# Without QGD and Using Quantum Walks

In [8]:
import os
import numpy as np
import pandas as pd
import json
import cirq
import matplotlib.pyplot as plt
from scipy.optimize import minimize

# Step 1: Load Historical Stock Data from JSON File
def load_historical_stock_data(filename):
    with open(filename, 'r') as file:
        data = json.load(file)
    stocks = list(data.keys())[15:25]  # Use only the first 10 stocks
    prices = []
    for stock in stocks:
        prices.append([entry['close'] for entry in data[stock]])
    prices = np.array(prices).T  # Transpose to get days as rows and stocks as columns
    return pd.DataFrame(prices, columns=stocks)

# Load historical stock prices from 'historical.json'
stock_prices = load_historical_stock_data('historical.json')
num_stocks = stock_prices.shape[1]
num_days = stock_prices.shape[0]

# Normalize stock prices to bring them to the same scale
normalized_stock_prices = stock_prices / stock_prices.iloc[0]

# Step 2: Define the Cost Function with Risk Factor
def cost(weights, returns, risk_factors, risk_tolerance):
    portfolio_return = np.dot(weights, returns.mean(axis=0))
    portfolio_risk = np.sqrt(np.dot(weights.T, np.dot(np.cov(returns, rowvar=False), weights)))
    weighted_risk_factor = np.dot(weights, risk_factors)
    
    # Adjust the risk penalty based on user-defined risk tolerance
    risk_penalty = portfolio_risk * risk_tolerance
    return -portfolio_return + risk_penalty + 0.1 * weighted_risk_factor
  # Minimize negative return + risk + risk factor

# Step 3: Construct Adjacency Matrix for Stock Correlations
def construct_adjacency_matrix(returns):
    covariance_matrix = np.cov(returns, rowvar=False)
    max_cov = np.max(np.abs(covariance_matrix))
    if max_cov != 0:
        adjacency_matrix = covariance_matrix / max_cov  # Normalize covariance values to [-1, 1]
    else:
        adjacency_matrix = covariance_matrix
    # Set diagonal to zero to avoid self-loops
    np.fill_diagonal(adjacency_matrix, 0)
    return adjacency_matrix

# Step 4: Define Quantum Walk Circuit on Correlation Graph
def quantum_walk_on_graph(qubits, adjacency_matrix, steps):
    circuit = cirq.Circuit()
    num_qubits = len(qubits)
    
    # Define coin qubits
    coin_qubits = [cirq.NamedQubit(f'coin_{i}') for i in range(num_qubits)]
    
    # Initialize coin and position in superposition
    for qubit in qubits + coin_qubits:
        circuit.append(cirq.H(qubit))
    
    # Quantum walk steps
    for _ in range(steps):
        # Coin operator: Apply Hadamard to coin qubits
        for coin in coin_qubits:
            circuit.append(cirq.H(coin))
        
        # Shift operator: Based on adjacency matrix
        for i in range(num_qubits):
            for j in range(num_qubits):
                if adjacency_matrix[i, j] != 0 and i != j:
                    # Conditional shift using coin qubits
                    circuit.append(cirq.CCX(coin_qubits[i], qubits[i], qubits[j]))
    
    # Measure position qubits
    circuit.append(cirq.measure(*qubits, key='position'))
    return circuit

# Step 5: Optimize Portfolio with Quantum Walk
def optimize_portfolio_with_quantum_walk(stock_prices, total_investment, steps=3):
    returns = stock_prices.pct_change().dropna().values
    num_stocks = returns.shape[1]

    # Calculate risk factors based on fluctuation percentage
    fluctuation = stock_prices.pct_change().std().values
    risk_factors = fluctuation / np.max(fluctuation)  # Normalize risk factors

    # Print risk factors
    print("Risk Factors for each stock:")
    for stock, risk in zip(stock_prices.columns, risk_factors):
        print(f"{stock}: {risk:.2f}")

    # Construct adjacency matrix for stock correlations
    adjacency_matrix = construct_adjacency_matrix(returns)

    # Initialize qubits for position and coin
    qubits = [cirq.NamedQubit(f'pos_{i}') for i in range(num_stocks)]

    # Build the quantum walk circuit
    circuit = quantum_walk_on_graph(qubits, adjacency_matrix, steps)

    # Simulate the circuit
    simulator = cirq.Simulator()
    result = simulator.run(circuit, repetitions=1000)
    measurements = result.measurements['position']

    # Process measurements to get probabilities for each stock
    position_counts = {}
    for measurement in measurements:
        bitstring = ''.join(str(bit) for bit in measurement)
        index = int(bitstring, 2)
        if index < num_stocks:
            position_counts[index] = position_counts.get(index, 0) + 1

    # Map indices to weights
    sampled_weights = np.zeros(num_stocks)
    for index, count in position_counts.items():
        sampled_weights[index] = count

    # Normalize the weights
    if np.sum(sampled_weights) == 0:
        sampled_weights = np.ones(num_stocks) / num_stocks
    else:
        sampled_weights = sampled_weights / np.sum(sampled_weights)

    # Use the sampled weights to calculate investment amounts
    investment_amounts = sampled_weights * total_investment

    # Evaluate the cost of the sampled portfolio
    portfolio_cost = cost(sampled_weights, returns, risk_factors, risk_tolerance=0.5)
    print(f"Portfolio Cost from Quantum Walk: {portfolio_cost:.4f}")

    return sampled_weights, investment_amounts

# Execute Optimization with Total Investment
total_investment = 10000  # Total investment amount
optimized_weights, investment_amounts = optimize_portfolio_with_quantum_walk(stock_prices, total_investment, steps=3)

# Create 'graphs' folder if it doesn't exist
if not os.path.exists('graphs_corrected'):
    os.makedirs('graphs_corrected')

# Step 7: Visualize Results

# 1. Line graph of normalized historical stock prices
plt.figure(figsize=(12, 6))
plt.plot(normalized_stock_prices)
plt.title('Normalized Historical Stock Prices Over Time')
plt.xlabel('Days')
plt.ylabel('Normalized Price')
plt.legend(stock_prices.columns, bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid()
plt.tight_layout()
plt.savefig('graphs_corrected/normalized_historical_stock_prices.png')
plt.close()

# 2. Histogram of sampled weights from quantum walk
plt.figure(figsize=(12, 6))
plt.bar(stock_prices.columns, optimized_weights)
plt.title('Sampled Weights from Quantum Walk')
plt.xlabel('Stocks')
plt.ylabel('Normalized Weight')
plt.xticks(rotation=45, ha='right')
plt.grid()
plt.tight_layout()
plt.savefig('graphs_corrected/quantum_walk_weights.png')
plt.close()

# 3. Pie chart of final investment amounts
plt.figure(figsize=(8, 8))
plt.pie(investment_amounts, labels=stock_prices.columns, autopct='%1.1f%%', startangle=140)
plt.title('Final Investment Distribution')
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.tight_layout()
plt.savefig('graphs_corrected/investment_distribution.png')
plt.close()

Risk Factors for each stock:
SBI LIFE INSURANCE CO LTD: 0.56
BAJAJ FINANCE LIMITED: 0.59
LARSEN & TOUBRO LTD.: 0.65
BRITANNIA INDUSTRIES LTD: 0.44
BAJAJ AUTO LIMITED: 0.57
NTPC LTD: 0.77
TITAN COMPANY LIMITED: 0.53
ADANI ENTERPRISES LIMITED: 0.98
ADANI PORT & SEZ LTD: 1.00
SUN PHARMACEUTICAL IND L: 0.43
Portfolio Cost from Quantum Walk: 0.0768


# with QGD instead of Quantum Walks

In [9]:
import os
import numpy as np
import pandas as pd
import json
import cirq
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import pennylane as qml
from pennylane import numpy as pnp

# Step 1: Load Historical Stock Data from JSON File
def load_historical_stock_data(filename):
    with open(filename, 'r') as file:
        data = json.load(file)
    stocks = list(data.keys())[15:25]  # Use only the first 10 stocks
    prices = []
    for stock in stocks:
        prices.append([entry['close'] for entry in data[stock]])
    prices = np.array(prices).T  # Transpose to get days as rows and stocks as columns
    return pd.DataFrame(prices, columns=stocks)

# Load historical stock prices from 'historical.json'
stock_prices = load_historical_stock_data('historical.json')
num_stocks = stock_prices.shape[1]
num_days = stock_prices.shape[0]

# Normalize stock prices to bring them to the same scale
normalized_stock_prices = stock_prices / stock_prices.iloc[0]

# Step 2: Define the Cost Function with Risk Factor
def cost(weights, returns, risk_factors, risk_tolerance):
    portfolio_return = np.dot(weights, returns.mean(axis=0))
    portfolio_risk = pnp.sqrt(pnp.dot(weights.T, pnp.dot(np.cov(returns, rowvar=False), weights)))
    weighted_risk_factor = np.dot(weights, risk_factors)
    
    # Adjust the risk penalty based on user-defined risk tolerance
    risk_penalty = portfolio_risk * risk_tolerance
    return -portfolio_return + risk_penalty + 0.1 * weighted_risk_factor

# Step 3: Construct Adjacency Matrix for Stock Correlations
def construct_adjacency_matrix(returns):
    covariance_matrix = np.cov(returns, rowvar=False)
    max_cov = np.max(covariance_matrix)
    if max_cov != 0:
        adjacency_matrix = covariance_matrix / max_cov  # Normalize covariance values to [0, 1]
    else:
        adjacency_matrix = covariance_matrix
    return adjacency_matrix

# Step 4: Define Quantum Walk Circuit on Correlation Graph
def quantum_walk_on_graph(qubits, adjacency_matrix, steps):
    circuit = cirq.Circuit()
    num_qubits = len(qubits)

    # Create Hadamard gates to initialize superposition
    for qubit in qubits:
        circuit.append(cirq.H(qubit))
    
    # Perform the quantum walk based on adjacency matrix
    for _ in range(steps):
        for i in range(num_qubits):
            for j in range(num_qubits):
                if i != j and adjacency_matrix[i, j] > 0:
                    circuit.append(cirq.CZ(qubits[i], qubits[j]) ** adjacency_matrix[i, j])  # Controlled phase shift based on correlation
        for i in range(num_qubits):
            circuit.append(cirq.H(qubits[i]))  # Mixing operation

    # Measurement
    circuit.append(cirq.measure(*qubits, key='result'))
    return circuit

# Step 5: Define Quantum Cost Function with PennyLane
def quantum_cost(weights, returns, risk_factors, risk_tolerance):
    num_qubits = len(weights)
    dev = qml.device("default.qubit", wires=num_qubits)

    @qml.qnode(dev)
    def circuit(weights):
        # Apply rotation gates based on weights
        for i in range(num_qubits):
            qml.RY(weights[i], wires=i)

        # Entanglement with CNOT gates
        for i in range(num_qubits - 1):
            qml.CNOT(wires=[i, i + 1])

        return [qml.expval(qml.PauliZ(i)) for i in range(num_qubits)]

    def cost_function(weights):
        quantum_expvals = circuit(weights)
        portfolio_return = np.dot(quantum_expvals, returns.mean(axis=0))
        portfolio_risk = pnp.sqrt(pnp.dot(weights.T, pnp.dot(np.cov(returns, rowvar=False), weights)))
        weighted_risk_factor = np.dot(weights, risk_factors)
        risk_penalty = portfolio_risk * risk_tolerance
        return -portfolio_return + risk_penalty + 0.1 * weighted_risk_factor

    return cost_function

# Step 6: Quantum Gradient Descent Optimization
def optimize_portfolio_with_qgd(stock_prices, total_investment, steps=3):
    returns = stock_prices.pct_change().dropna().values
    num_stocks = returns.shape[1]

    # Calculate risk factors based on fluctuation percentage
    fluctuation = stock_prices.pct_change().std().values
    risk_factors = fluctuation / np.max(fluctuation)  # Normalize risk factors

    # Print risk factors
    print("Risk Factors for each stock:")
    for stock, risk in zip(stock_prices.columns, risk_factors):
        print(f"{stock}: {risk:.2f}")

    # Define the quantum cost function
    # np.random.seed(42)
    quantum_cost_function = quantum_cost(weights=np.random.rand(num_stocks), returns=returns, risk_factors=risk_factors, risk_tolerance=0.5)

    # Use quantum natural gradient descent from PennyLane to optimize the weights
    opt = qml.GradientDescentOptimizer(stepsize=0.01)
    weights = pnp.random.rand(num_stocks, requires_grad=True)
    max_iterations = 100

    for i in range(max_iterations):
        weights = opt.step(quantum_cost_function, weights)
        if (i + 1) % 10 == 0:
            current_cost = quantum_cost_function(weights)
            print(f"Iteration {i + 1}: Cost = {current_cost:.4f}")

    final_weights = weights / pnp.sum(weights)  # Normalize weights
    investment_amounts = final_weights * total_investment

    # Ensure no negative investments
    investment_amounts = pnp.maximum(investment_amounts, 0)
    return final_weights, investment_amounts

# Execute Optimization with Total Investment
total_investment = 10000  # Total investment amount
optimized_weights, investment_amounts = optimize_portfolio_with_qgd(stock_prices, total_investment, steps=3)

# Create 'graphs' folder if it doesn't exist
if not os.path.exists('graphs_new_test3'):
    os.makedirs('graphs_new_test3')

# Step 7: Visualize Results

# 1. Line graph of normalized historical stock prices
plt.figure(figsize=(12, 6))
plt.plot(normalized_stock_prices)
plt.title('Normalized Historical Stock Prices Over Time')
plt.xlabel('Days')
plt.ylabel('Normalized Price')
plt.legend(stock_prices.columns, bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid()
plt.tight_layout()
plt.savefig('graphs_new_test3/normalized_historical_stock_prices.png')
plt.close()

# 2. Pie chart of final investment amounts
plt.figure(figsize=(8, 8))
plt.pie(investment_amounts, labels=stock_prices.columns, autopct='%1.1f%%', startangle=140)
plt.title('Final Investment Distribution with QGD')
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.tight_layout()
plt.savefig('graphs_new_test3/investment_distribution.png')
plt.close()

# 3. Bar graph of optimized portfolio weights
plt.figure(figsize=(12, 6))
plt.bar(stock_prices.columns, optimized_weights)
plt.title('Optimized Portfolio Weights')
plt.xlabel('Stocks')
plt.ylabel('Weights')
plt.xticks(rotation=45, ha='right')
plt.grid()
plt.tight_layout()
plt.savefig('graphs_new_test3/optimized_portfolio_weights.png')
plt.close()



Risk Factors for each stock:
SBI LIFE INSURANCE CO LTD: 0.56
BAJAJ FINANCE LIMITED: 0.59
LARSEN & TOUBRO LTD.: 0.65
BRITANNIA INDUSTRIES LTD: 0.44
BAJAJ AUTO LIMITED: 0.57
NTPC LTD: 0.77
TITAN COMPANY LIMITED: 0.53
ADANI ENTERPRISES LIMITED: 0.98
ADANI PORT & SEZ LTD: 1.00
SUN PHARMACEUTICAL IND L: 0.43
Iteration 10: Cost = 0.3593
Iteration 20: Cost = 0.3540
Iteration 30: Cost = 0.3486
Iteration 40: Cost = 0.3433
Iteration 50: Cost = 0.3380
Iteration 60: Cost = 0.3326
Iteration 70: Cost = 0.3273
Iteration 80: Cost = 0.3220
Iteration 90: Cost = 0.3167
Iteration 100: Cost = 0.3113
